In [21]:
import pandas as pd
from tqdm import tqdm
import statsmodels.api as sm
from sodapy import Socrata
import numpy as np
# Retrieve all the data using SOAP API methods
client = Socrata("opendata.utah.gov", None)
results = client.get("herb-zqda", limit=300000)
df = pd.DataFrame.from_records(results)

# Due to the sheer size of the dataset, we took a random sample of the data to create models. 
df = df.sample(frac = .06)
# Separate the crash_datetime column into date and time

df['year'] = pd.DatetimeIndex(df['crash_datetime']).year
df['month'] = pd.DatetimeIndex(df['crash_datetime']).month
df['day'] = pd.DatetimeIndex(df['crash_datetime']).day
df['hour'] = pd.DatetimeIndex(df['crash_datetime']).hour
df.drop(columns=['crash_datetime','milepoint'], inplace=True)

df


,crash_id,route,lat_utm_y,long_utm_x,main_road_name,city,county_name,crash_severity_id,work_zone_related,pedestrian_involved,...,older_driver_involved,night_dark_condition,single_vehicle,distracted_driving,drowsy_driving,roadway_departure,year,month,day,hour
167345,11057723,126,4553982.986,413884.5998,1300 N MAIN ST,SUNSET,DAVIS,2,False,False,...,True,False,False,False,False,False,2018,5,1,15
164475,10999996,15,4558294.017,414462.8837,I-15,RIVERDALE,WEBER,2,False,False,...,False,False,False,False,False,False,2017,9,29,17
136737,11098664,2068,4486401.019,427715.5696,1300 E,DRAPER,SALT LAKE,1,False,False,...,False,False,False,False,False,False,2018,9,26,8
73740,11238205,111,4503514.584,407724.3627,4283 S,WEST VALLEY CITY,SALT LAKE,1,False,False,...,False,False,False,False,False,False,2019,11,29,14
128833,10884119,3347,4556887.178,421298.3793,WASATCH DR,SOUTH OGDEN,WEBER,2,False,False,...,False,False,False,False,False,False,2016,9,22,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62785,11147709,15,4490501.3,423892.3117,I-15,SANDY,SALT LAKE,1,False,False,...,False,False,False,False,False,False,2019,2,11,7
47562,11065271,152,4498664.916,429452.851,6331 S,MURRAY,SALT LAKE,2,False,False,...,True,False,False,False,False,False,2018,5,28,18
80691,10861374,89,4622052,438584,SR-89,OUTSIDE CITY LIMITS,CACHE,3,False,False,...,True,False,False,False,False,False,2016,6,22,14
49198,11075873,15,4488720.941,424218.2832,I-15,SANDY,SALT LAKE,2,False,False,...,False,False,False,False,False,False,2018,7,6,16


In [22]:
# Remove any rows with missing data
df2 = df.dropna(axis=0, how='any')
# Cast columns to correct datatype
df2['year'] = df2['year'].astype(str)
df2['month']= df2['month'].astype(str)
df2['day'] = df2['day'].astype(str)
df2['hour']=df2['hour'].astype(str)
df2['crash_severity_id']=  df2['crash_severity_id'].astype(int)
df2['crash_id']=df2['crash_id'].astype(int)
df2['lat_utm_y']=df2['lat_utm_y'].astype(float)
df2['long_utm_x']=df2['long_utm_x'].astype(float)




df2.drop(columns=['main_road_name', 'crash_id'], inplace=True)
# Create dummies for catagarical variables
for col in df2:
    if not pd.api.types.is_numeric_dtype(df2[col]):
          df2 = pd.get_dummies(df2, columns=[col], prefix=col, drop_first=True)
df2.head()

C:\Users\colep\AppData\Local\Temp/ipykernel_29272/1545305476.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['year'] = df2['year'].astype(str)
C:\Users\colep\AppData\Local\Temp/ipykernel_29272/1545305476.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['month']= df2['month'].astype(str)
C:\Users\colep\AppData\Local\Temp/ipykernel_29272/1545305476.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

,lat_utm_y,long_utm_x,crash_severity_id,route_10,route_10000,route_1005,route_1006,route_101,route_1018,route_102,...,hour_21,hour_22,hour_23,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9
167345,4553982.986,413884.5998,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
164475,4558294.017,414462.8837,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
136737,4486401.019,427715.5696,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
73740,4503514.584,407724.3627,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
128833,4556887.178,421298.3793,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
# Set x and y variables
y =np.log(df2['crash_severity_id'])
X = df2.select_dtypes(np.number).assign(const=1)
X = X.drop(columns=['crash_severity_id'])

# Run the multiple linear regression model
model = sm.OLS(y, X).fit()

# View results
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:      crash_severity_id   R-squared:                       0.256
Model:                            OLS   Adj. R-squared:                  0.177
Method:                 Least Squares   F-statistic:                     3.236
Date:                Fri, 08 Apr 2022   Prob (F-statistic):          3.87e-263
Time:                        00:57:50   Log-Likelihood:                -5838.1
No. Observations:               14655   AIC:                         1.450e+04
Df Residuals:                   13245   BIC:                         2.520e+04
Df Model:                        1409                                         
Covariance Type:            nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
lat_

In [24]:
# Get and remove all the features that have a p-value above .05
import lxml
testDf = pd.read_html(model.summary().tables[1].as_html(),header=0,index_col=0)[0]
filteredDf = testDf[testDf['P>|t|'] > .05] 
print(filteredDf)
index = filteredDf.index
df3 = df2.copy()


for i in index:
    if i != 'const':
        df3.drop(columns=i, inplace=True)

df3

                     coef       std err      t  P>|t|    [0.025        0.975]
lat_utm_y   -2.926000e-07  5.840000e-07 -0.501  0.616 -0.000001  8.510000e-07
long_utm_x  -7.198000e-07  5.700000e-07 -1.263  0.206 -0.000002  3.970000e-07
route_10     4.240000e-01  4.240000e-01  1.000  0.317 -0.407000  1.255000e+00
route_10000  1.907000e-01  5.590000e-01  0.341  0.733 -0.906000  1.287000e+00
route_1005   3.017000e-01  5.600000e-01  0.539  0.590 -0.796000  1.399000e+00
...                   ...           ...    ...    ...       ...           ...
hour_6       2.070000e-02  3.400000e-02  0.618  0.536 -0.045000  8.700000e-02
hour_7       4.420000e-02  3.200000e-02  1.379  0.168 -0.019000  1.070000e-01
hour_8       4.230000e-02  3.200000e-02  1.324  0.185 -0.020000  1.050000e-01
hour_9      -7.300000e-03  3.300000e-02 -0.221  0.825 -0.072000  5.700000e-02
const        1.464000e+00  2.515000e+00  0.582  0.561 -3.467000  6.395000e+00

[1382 rows x 6 columns]


,crash_severity_id,route_1094,route_1186,route_1201,route_1312,route_170000,route_1768,route_2165,route_2340,route_2644,...,older_driver_involved_True,single_vehicle_True,distracted_driving_True,drowsy_driving_True,roadway_departure_True,month_10,month_3,month_7,month_8,hour_1
167345,2,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
164475,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
136737,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
73740,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
128833,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62785,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47562,2,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
80691,3,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
49198,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [25]:
# Rerun Model
# Set x and y variables
y2 = np.log(df3['crash_severity_id'])
X2 = df3.select_dtypes(np.number).assign(const=1)  # drop all categorical features and allow y-intercept to vary
X2 = X2.drop(columns=['crash_severity_id'])

# Run the multiple linear regression model
model2 = sm.OLS(y2, X2).fit()

# View results
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:      crash_severity_id   R-squared:                       0.171
Model:                            OLS   Adj. R-squared:                  0.168
Method:                 Least Squares   F-statistic:                     65.54
Date:                Fri, 08 Apr 2022   Prob (F-statistic):               0.00
Time:                        00:57:59   Log-Likelihood:                -6630.8
No. Observations:               14655   AIC:                         1.336e+04
Df Residuals:                   14608   BIC:                         1.371e+04
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
route_1094          

In [26]:
# Once again, get and remove all features with a p-value over .05
testDf2 = pd.read_html(model2.summary().tables[1].as_html(),header=0,index_col=0)[0]
filteredDf2 = testDf2[testDf2['P>|t|'] > .05] 
print(filteredDf2)
index2 = filteredDf2.index

for i in index2:
    if i != 'const':
        df3.drop(columns=i, inplace=True)



                          coef  std err      t  P>|t|  [0.025  0.975]
route_170000            0.4979    0.311  1.598  0.110  -0.113   1.108
route_330000            0.6331    0.382  1.656  0.098  -0.116   1.383
city_BRYCE CANYON CITY -0.4605    0.311 -1.479  0.139  -1.071   0.150
city_ESCALANTE         -0.5301    0.382 -1.389  0.165  -1.278   0.218
county_name_SAN JUAN    0.0419    0.052  0.798  0.425  -0.061   0.145
month_10                0.0154    0.011  1.350  0.177  -0.007   0.038
month_3                 0.0200    0.012  1.690  0.091  -0.003   0.043
month_7                 0.0176    0.012  1.471  0.141  -0.006   0.041


In [27]:
# Rerun Model
# Set x and y variables
y3 = np.log(df3['crash_severity_id'])
X3 = df3.select_dtypes(np.number).assign(const=1)  # drop all categorical features and allow y-intercept to vary
X3 = X3.drop(columns=['crash_severity_id'])

# Run the multiple linear regression model
model3 = sm.OLS(y3, X3).fit()

# View results
print(model3.summary())

                            OLS Regression Results                            
Dep. Variable:      crash_severity_id   R-squared:                       0.170
Model:                            OLS   Adj. R-squared:                  0.168
Method:                 Least Squares   F-statistic:                     78.94
Date:                Fri, 08 Apr 2022   Prob (F-statistic):               0.00
Time:                        00:57:59   Log-Likelihood:                -6637.8
No. Observations:               14655   AIC:                         1.335e+04
Df Residuals:                   14616   BIC:                         1.365e+04
Df Model:                          38                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
route_1094          

In [28]:
# Once again, get and remove all features with a p-value over .05
testDf = pd.read_html(model3.summary().tables[1].as_html(),header=0,index_col=0)[0]
filteredDf = testDf[testDf['P>|t|'] > .05] 
print(filteredDf)
index = filteredDf.index

for i in index:
    if i != 'const':
        df3.drop(columns=i, inplace=True)


Empty DataFrame
Columns: [coef, std err, t, P>|t|, [0.025, 0.975]]
Index: []


In [29]:
# Rerun Model
# Set x and y variables
y3 = np.log(df3['crash_severity_id'])
X3 = df3.select_dtypes(np.number).assign(const=1)  # drop all categorical features and allow y-intercept to vary
X3 = X3.drop(columns=['crash_severity_id'])

# Run the multiple linear regression model
model4 = sm.OLS(y3, X3).fit()

# View results
print(model4.summary())

# After multiple runs of removing p-values, this model is the first model that has no features with a p-value over .05. 
# In addition, the F-statistic has increased as we remove the features, showing greater model fit.

                            OLS Regression Results                            
Dep. Variable:      crash_severity_id   R-squared:                       0.170
Model:                            OLS   Adj. R-squared:                  0.168
Method:                 Least Squares   F-statistic:                     78.94
Date:                Fri, 08 Apr 2022   Prob (F-statistic):               0.00
Time:                        00:57:59   Log-Likelihood:                -6637.8
No. Observations:               14655   AIC:                         1.335e+04
Df Residuals:                   14616   BIC:                         1.365e+04
Df Model:                          38                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
route_1094          

In [30]:
# We have a skewness of 1.5, so we are going to transform the label to try and reduce skewness
y3 = np.log(df3['crash_severity_id'])

# Rerun model
model4 = sm.OLS(y3, X3).fit()

# View results
print(model4.summary())

                            OLS Regression Results                            
Dep. Variable:      crash_severity_id   R-squared:                       0.170
Model:                            OLS   Adj. R-squared:                  0.168
Method:                 Least Squares   F-statistic:                     78.94
Date:                Fri, 08 Apr 2022   Prob (F-statistic):               0.00
Time:                        00:58:00   Log-Likelihood:                -6637.8
No. Observations:               14655   AIC:                         1.335e+04
Df Residuals:                   14616   BIC:                         1.365e+04
Df Model:                          38                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
route_1094          

In [31]:
# Create a blank DataFrame to store the results
df_vif = pd.DataFrame(columns=['VIF', 'Tolerance'])
df3 = pd.get_dummies(df3, columns=df3.select_dtypes(include=['object']).columns, drop_first=True)

for col in df3.drop(columns=['crash_severity_id']):
    y = df3[col]
    X = df3.drop(columns=[col, 'crash_severity_id']).assign(const=1)
    
    r_squared = sm.OLS(y, X).fit().rsquared

    if r_squared < 1: # Prevent division by zero runtime error
        df_vif.loc[col] = [1/(1 - r_squared), 1 - r_squared]
    else:
        df_vif.loc[col] = ['infinity', 1 - r_squared]

df_vif['VIF']=df_vif['VIF'].astype(float)
df_vif['Tolerance']=df_vif['Tolerance'].astype(float)
df_vif.sort_values(by=['VIF'], ascending=False)

,VIF,Tolerance
single_vehicle_True,3.547291,0.281905
roadway_departure_True,2.813907,0.355378
wild_animal_related_True,1.670506,0.598621
pedestrian_involved_True,1.213815,0.823848
overturn_rollover_True,1.145257,0.873166
bicyclist_involved_True,1.137877,0.878830
intersection_related_True,1.117897,0.894537
dui_True,1.053860,0.948892
drowsy_driving_True,1.041743,0.959930
unrestrained_True,1.028950,0.971864


In [32]:
# Rerun Model after dropping city_JENSEN and route_149 (they have bad VIF values)
# Set x and y variables
class_df = df3.copy()
finaly = np.log(df3['crash_severity_id'])
X = df3.select_dtypes(np.number).assign(const=1)  # drop all categorical features and allow y-intercept to vary
X = X.drop(columns=['crash_severity_id'])

# Run the multiple linear regression model
model5 = sm.OLS(finaly, X).fit()

# View results
print(model5.summary())

                            OLS Regression Results                            
Dep. Variable:      crash_severity_id   R-squared:                       0.170
Model:                            OLS   Adj. R-squared:                  0.168
Method:                 Least Squares   F-statistic:                     78.94
Date:                Fri, 08 Apr 2022   Prob (F-statistic):               0.00
Time:                        00:58:01   Log-Likelihood:                -6637.8
No. Observations:               14655   AIC:                         1.335e+04
Df Residuals:                   14616   BIC:                         1.365e+04
Df Model:                          38                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
route_1094          

In [33]:
# The following code will look for the best type of linear regression for our model, so we can consider the best model for our analysis.
fit = {}    # Use this to store each of the fit metrics
models = {} # Use this to store each of the models
        
# 1. LINEAR MODELS: Assumes normal distribution, homoscedasticity, no multicollinearity, independence, and no auto-correlation (some exceptions apply; some of these algorithms are better at handling violations of these assumptions).
import sklearn.linear_model as lm
from sklearn.model_selection import KFold, cross_val_score
from numpy import mean

# We will automate this later, just create
y = df2['crash_severity_id']
X = df2.drop(columns=['crash_severity_id'])


# Set up a standard cross_validation object to use for each algorithm
cv = KFold(n_splits=5, random_state=12345, shuffle=True)

# 1.1. Ordinary Least Squares Multiple Linear Regression
model_ols = lm.LinearRegression()
fit['OLS'] = mean(cross_val_score(model_ols, X, np.log(y), scoring='r2', cv=cv, n_jobs=-1))
models['OLS'] = model_ols

# 1.2. Ridge Regression: More robust to multicollinearity
model_rr = lm.Ridge(alpha=0.5) # adjust this alpha parameter for better results (between 0 and 1)
fit['Ridge'] = mean(cross_val_score(model_rr, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['Ridge'] = model_rr

# 1.3. Lasso Regression: Better for sparse values like RetweetCount where most are zeros but a few have many retweets.
model_lr = lm.Lasso(alpha=0.1) # adjust this alpha parameter for better results (between 0 and 1)
fit['Lasso'] = mean(cross_val_score(model_lr, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['Lasso'] = model_lr

# 1.4. Least Angle Regression: Good when the number of features is greater than the number of samples.
model_llr = lm.LassoLars(alpha=0.1) # adjust this alpha parameter for better results (between 0 and 1)
fit['LARS'] = mean(cross_val_score(model_llr, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['LARS'] = model_llr

# 1.5. Bayesian Regression: Probability based and allows regularization parameters, automatically tuned to data.
model_br = lm.BayesianRidge()
fit['Bayesian'] = mean(cross_val_score(model_br, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['Bayesian'] = model_br

# 1.6. Generalized Linear Regression (Poisson): Good for non-normal distribution, count-based data, and a Poisson distribution.
model_pr = lm.TweedieRegressor(power=1, link="log") # Power=1 means this is a Poisson
fit['Poisson'] = mean(cross_val_score(model_pr, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['Poisson'] = model_pr

# 1.7. Generalized Linear Regression (Gamma): Good for non-normal distribution, continuous data, and a Gamma distribution.
model_gr = lm.TweedieRegressor(power=2, link="log") # Power=2 means this is a Gamma
fit['Gamma'] = mean(cross_val_score(model_gr, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['Gamma'] = model_gr

# 1.8. Generalized Linear Regression (Inverse Gamma): Good non-normal distribution, continuous data, and an inverse Gamma distribution.
model_igr = lm.TweedieRegressor(power=3) # Power=3 means this is an inverse Gamma
fit['Inverse'] = mean(cross_val_score(model_igr, X, y, scoring='r2', cv=cv, n_jobs=-1))
models['Inverse'] = model_igr


# ----------------------------------------------------------------------------------------------------
# Sort and print the dictionary by greatest R squared to least
df_fit = pd.DataFrame({'R-squared':fit})
df_fit.sort_values(by=['R-squared'], ascending=False)

C:\Users\colep\is455\venv455\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\colep\is455\venv455\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\colep\is455\venv455\lib\site-packages\sklearn\linear_model\_glm\glm.py", line 310, in fit
    opt_res = scipy.optimize.minimize(
  File "C:\Users\colep\is455\venv455\lib\site-packages\scipy\optimize\_minimize.py", line 623, in minimize
    return _minimize_lbfgsb(fun, x0, args, jac, boun

,R-squared
Bayesian,0.170900
Ridge,0.135274
OLS,0.084126
Gamma,-0.000492
Poisson,-0.000492
LARS,-0.000492
Lasso,-0.000589
Inverse,NaN


In [34]:
# After looking at the features from the original dataset, we decided on the following features to use in a categorical model. These features will help regulators 
# make the most informed decisions.

# Make our dataframe from the original dataset
df_class_models = df.copy()
df_class_models.drop(columns=[ 'crash_id', 'route', 'lat_utm_y' ,'long_utm_x','main_road_name','city', 'work_zone_related','pedestrian_involved','bicyclist_involved',
 'motorcycle_involved','improper_restraint', 'intersection_related', 'wild_animal_related', 'domestic_animal_related', 'overturn_rollover', 'commercial_motor_veh_involved', 'single_vehicle'
 , 'distracted_driving', 'roadway_departure', 'year', 'month', 'day', 'hour'], inplace=True)

for col in df_class_models:
    if not pd.api.types.is_numeric_dtype(df_class_models[col]) and col != 'crash_severity_id':
        df_class_models = pd.get_dummies(df_class_models, columns=[col], prefix=col, drop_first=True)

       

In [35]:
# Here we decided to look at three different kinds of categorical analysis. We will look at the following regressions (seen in the code below):
# We have ran this function with all the features and data, but found that the r-squared doesn't change with the features we chose
# earlier.

df_class_models['crash_severity_id'] = df_class_models['crash_severity_id'].astype(str)

def fit_crossvalidate_clf(df, label, k=10, r=5, repeat=True):
  import sklearn.linear_model as lm, pandas as pd, sklearn.ensemble as se, numpy as np
  from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score
  from numpy import mean, std
  from sklearn import svm
  from sklearn import gaussian_process
  from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
  from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
  from sklearn import svm
  from sklearn.naive_bayes import CategoricalNB
  from xgboost import XGBClassifier
  from sklearn import preprocessing
  from sklearn.neural_network import MLPClassifier

  X = df.drop(columns=[label])
  y = df[label]

  if repeat:
    cv = RepeatedKFold(n_splits=k, n_repeats=r, random_state=12345)
  else:
    cv = KFold(n_splits=k, random_state=12345, shuffle=True)

  fit = {}    # Use this to store each of the fit metrics
  models = {} # Use this to store each of the models

  # Create the model objects
  model_log = lm.LogisticRegression(max_iter=100)
  model_logcv = lm.RidgeClassifier()
  model_sgd = lm.SGDClassifier(max_iter=1000, tol=1e-3)
  model_pa = lm.PassiveAggressiveClassifier(max_iter=1000, random_state=12345, tol=1e-3)
  model_per = lm.Perceptron(fit_intercept=False, max_iter=10, tol=None, shuffle=False)
  model_knn = KNeighborsClassifier(n_neighbors=3)
  model_svm = svm.SVC(decision_function_shape='ovo') # Remove the parameter for two-class model
  model_nb = CategoricalNB()
  model_bag = se.BaggingClassifier(KNeighborsClassifier(), max_samples=0.5, max_features=0.5)
  model_ada = se.AdaBoostClassifier(n_estimators=100, random_state=12345)
  model_ext = se.ExtraTreesClassifier(n_estimators=100, random_state=12345)
  model_rf = se.RandomForestClassifier(n_estimators=10)
  model_hgb = se.HistGradientBoostingClassifier(max_iter=100)
  model_vot = se.VotingClassifier(estimators=[('lr', model_log), ('rf', model_ext), ('gnb', model_hgb)], voting='hard')
  model_gb = se.GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
  estimators = [('ridge', lm.RidgeCV()), ('lasso', lm.LassoCV(random_state=12345)), ('knr', KNeighborsRegressor(n_neighbors=20, metric='euclidean'))]
  final_estimator = se.GradientBoostingRegressor(n_estimators=25, subsample=0.5, min_samples_leaf=25, max_features=1, random_state=12345)
  model_st = se.StackingRegressor(estimators=estimators, final_estimator=final_estimator)
  model_xgb = XGBClassifier()
  model_nn = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=12345)

  # Fit a cross-validated R squared score and add it to the dict
  fit['Logistic'] = mean(cross_val_score(model_log, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['Ridge'] = mean(cross_val_score(model_logcv, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['SGD'] = mean(cross_val_score(model_sgd, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['PassiveAggressive'] = mean(cross_val_score(model_pa, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['Perceptron'] = mean(cross_val_score(model_per, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['KNN'] = mean(cross_val_score(model_knn, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['SVM'] = mean(cross_val_score(model_svm, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['NaiveBayes'] = mean(cross_val_score(model_nb, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['Bagging'] = mean(cross_val_score(model_bag, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['AdaBoost'] = mean(cross_val_score(model_ada, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['ExtraTrees'] = mean(cross_val_score(model_ext, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['RandomForest'] = mean(cross_val_score(model_rf, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['HistGradient'] = mean(cross_val_score(model_hgb, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['Voting'] = mean(cross_val_score(model_vot, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['GradBoost'] = mean(cross_val_score(model_gb, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['XGBoost'] = mean(cross_val_score(model_xgb, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['NeuralN'] = mean(cross_val_score(model_nn, X, y, scoring='accuracy', cv=cv, n_jobs=-1))

  # Add the model to another dictionary; make sure the keys have the same names as the list above
  models['Logistic'] = model_log
  models['Ridge'] = model_logcv
  models['SGD'] = model_sgd
  models['PassiveAggressive'] = model_pa
  models['Perceptron'] = model_per
  models['KNN'] = model_knn
  models['SVM'] = model_svm
  models['NaiveBayes'] = model_nb
  models['Bagging'] = model_bag
  models['AdaBoost'] = model_ada
  models['ExtraTrees'] = model_ext
  models['RandomForest'] = model_rf
  models['HistGradient'] = model_hgb
  models['Voting'] = model_vot
  models['GradBoost'] = model_gb
  models['XGBoost'] = model_xgb
  models['NeuralN'] = model_nn

  # Add the fit dictionary to a new DataFrame, sort, extract the top row, use it to retrieve the model object from the models dictionary
  df_fit = pd.DataFrame({'Accuracy':fit})
  df_fit.sort_values(by=['Accuracy'], ascending=False, inplace=True)
  best_model = df_fit.index[0]
  print(df_fit)

  return models[best_model].fit(X, y)


# We used this function to make sure the features we used were statistically significant.
def fs_variance(df, label="", p=0.8):
  from sklearn.feature_selection import VarianceThreshold
  import pandas as pd

  if label != "":
    X = df.drop(columns=[label])
    
  sel = VarianceThreshold(threshold=(p * (1 - p)))
  sel.fit_transform(X)

  # Add the label back in after removing poor features
  return df[sel.get_feature_names_out()].join(df[label])

def dump_pickle(model, file_name):
  import pickle
  pickle.dump(model, open(file_name, "wb"))

# This is where we clean the data, and create our model.
dfCat = fs_variance(df_class_models, label="crash_severity_id", p=.05)
model = fit_crossvalidate_clf(df_class_models, "crash_severity_id", 5, 2)



                   Accuracy
Bagging            0.706832
Logistic           0.706766
Ridge              0.706733
NaiveBayes         0.706502
SVM                0.706436
Voting             0.706403
SGD                0.706304
HistGradient       0.706172
NeuralN            0.705710
AdaBoost           0.703762
XGBoost            0.703564
ExtraTrees         0.703333
RandomForest       0.701386
GradBoost          0.694290
KNN                0.679373
Perceptron         0.613102
PassiveAggressive  0.607459


In [36]:
print(model)

BaggingClassifier(base_estimator=KNeighborsClassifier(), max_features=0.5,
                  max_samples=0.5)


In [37]:
# This is where we save the best model
dump_pickle(model, 'class_model.sav')

In [38]:
# This is where we prepare the model for .onxx
def load_pickle(file_name):
    import pickle
    model = pickle.load(open(file_name, "rb"))
    return model
model = load_pickle('class_model.sav')
model.predict(df_class_models.drop(columns=['crash_severity_id']))

array(['1', '1', '1', ..., '1', '1', '1'], dtype=object)

In [39]:
# This is where we prepare the model for .onxx
import skl2onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx.common.data_types import Int64TensorType


In [40]:
# This is where we prepare the model for .onxx
initial_type = [('int_input', Int64TensorType([None, 36]))]
onnx = convert_sklearn(model, initial_types=initial_type)
with open("best_clf_model.onnx", "wb") as f:
    f.write(onnx.SerializeToString())

C:\Users\colep\is455\venv455\lib\site-packages\sklearn\utils\deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)
